In [4]:
%load_ext autoreload
%autoreload 2

In [16]:
from src.settings import UC_MERCED_DATA_DIRECTORY, RESULTS_DIRECTORY
from src.data.ucmerced_dataset import TripletDataModule
from src.experiments import create_path_if_not_exists, run_bovw_experiments
import os

image_size = 256
dm = TripletDataModule(UC_MERCED_DATA_DIRECTORY, image_size, 0.8, 100, augment=False, normalize=False, permute=True)
dm.setup(None)
train_dataset = dm.train_dataset
test_dataset = dm.val_dataset

output_sizes = [25, 50, 100, 150]
samples = [10000]
bovw_path = os.path.join(RESULTS_DIRECTORY, "bovw")
create_path_if_not_exists(bovw_path)
dataset_path = os.path.join(bovw_path, "uc_merced")
create_path_if_not_exists(dataset_path)

In [17]:
values, values_per_class, cluster_numbers, sample_numbers = run_bovw_experiments(train_dataset, test_dataset, output_sizes, samples, "UC Merced", dataset_path)

Encoding as BOVW: 100%|██████████| 420/420 [00:00<00:00, 1224.49it/s]


Clusters: 25, samples: 10000
Clusters: 50, samples: 10000
Clusters: 100, samples: 10000
Clusters: 150, samples: 10000


In [27]:
import numpy as np
values_per_class = [list(list(zip(*single_experiment))[1]) for single_experiment in values_per_class]
np.array(values_per_class2)

array([[0.39525316, 0.69341772, 0.79424051, 0.7743038 , 0.6143038 ,
        0.20575949, 0.65101266, 0.39234177, 0.68278481, 0.86044304,
        0.68689873, 0.6285443 , 0.68126582, 0.62917722, 0.72272152,
        0.53911392, 0.67443038, 0.63886076, 0.75234177, 0.76468354,
        0.74303797],
       [0.44829114, 0.69113924, 0.79974684, 0.77765823, 0.63037975,
        0.20411392, 0.64120253, 0.39879747, 0.69373418, 0.85031646,
        0.70613924, 0.59607595, 0.63158228, 0.5764557 , 0.71101266,
        0.5085443 , 0.67886076, 0.63310127, 0.74101266, 0.79303797,
        0.76      ],
       [0.48056962, 0.75139241, 0.80873418, 0.81797468, 0.56727848,
        0.19917722, 0.67841772, 0.33303797, 0.71259494, 0.8471519 ,
        0.69797468, 0.62012658, 0.60620253, 0.53259494, 0.72816456,
        0.47658228, 0.69025316, 0.64398734, 0.72487342, 0.81512658,
        0.75702532],
       [0.55759494, 0.73025316, 0.84113924, 0.83886076, 0.62405063,
        0.18411392, 0.67208861, 0.31208861, 0.7206329

In [38]:
import pandas as pd
class_names = dm.val_dataset.class_names
df = pd.DataFrame.from_dict({"clusters": cluster_numbers, "samples": sample_numbers, "anmrr": values})
values_per_class_without_labels = [list(list(zip(*single_experiment))[1]) for single_experiment in values_per_class]
full_df = pd.concat([df, pd.DataFrame(np.array(values_per_class_without_labels), columns=class_names)], axis=1)
results_long_form = full_df.melt(id_vars=['clusters', 'samples', 'anmrr'], var_name='class', value_name='anmrr_per_class')
results_long_form['experiment_name'] = results_long_form.apply(lambda row: str(row['clusters']) + "_" + str(row['samples']), axis=1)

In [39]:
df

,clusters,samples,anmrr
0,25,10000,0.644045
1,50,10000,0.641486
2,100,10000,0.642345
3,150,10000,0.647456


In [34]:
np.array(values_per_class2).shape

(4, 21)

In [44]:
xd = pd.concat([df, pd.DataFrame(np.array(values_per_class2), columns=class_names)], axis=1)

In [58]:
import plotly.express as px

ble = xd.melt(id_vars=['clusters', 'samples', 'anmrr'], var_name='class', value_name='anmrr_per_class')
ble['experiment_name'] = ble.apply(lambda row: str(row['clusters']) + "_" + str(row['samples']), axis=1)

,clusters,samples,anmrr,class,anmrr_per_class,experiment_name
0,25,10000,0.644045,agricultural,0.395253,25_10000
1,50,10000,0.641486,agricultural,0.448291,50_10000
2,100,10000,0.642345,agricultural,0.480570,100_10000
3,150,10000,0.647456,agricultural,0.557595,150_10000
4,25,10000,0.644045,airplane,0.693418,25_10000
...,...,...,...,...,...,...
79,150,10000,0.647456,storagetanks,0.831203,150_10000
80,25,10000,0.644045,tenniscourt,0.743038,25_10000
81,50,10000,0.641486,tenniscourt,0.760000,50_10000
82,100,10000,0.642345,tenniscourt,0.757025,100_10000


In [54]:
ble

,clusters,samples,anmrr,class,anmrr_per_class,experiment_name
0,25,10000,0.644045,agricultural,0.395253,2510000
1,50,10000,0.641486,agricultural,0.448291,5010000
2,100,10000,0.642345,agricultural,0.480570,10010000
3,150,10000,0.647456,agricultural,0.557595,15010000
4,25,10000,0.644045,airplane,0.693418,NaN
...,...,...,...,...,...,...
79,150,10000,0.647456,storagetanks,0.831203,NaN
80,25,10000,0.644045,tenniscourt,0.743038,NaN
81,50,10000,0.641486,tenniscourt,0.760000,NaN
82,100,10000,0.642345,tenniscourt,0.757025,NaN


In [63]:
fig = px.line(ble, x='class', y='anmrr_per_class', color=['clusters', 'samples'])
fig.show(renderer='browser')

ValueError: All arguments should have the same length. The length of argument `color` is 2, whereas the length of  previously-processed arguments ['class', 'anmrr_per_class'] is 84

In [1]:
import pandas as pd
df = pd.read_pickle("..\\results\\bovw\\pattern_net\\results_PatternNet.pkl.gz")

In [2]:
df

,clusters,samples,anmrr,class,anmrr_per_class,experiment_name
0,25,10000,0.646608,airplane,0.794673,25_10000
1,50,10000,0.639308,airplane,0.793991,50_10000
2,100,10000,0.639503,airplane,0.816029,100_10000
3,150,10000,0.641029,airplane,0.810513,150_10000
4,25,10000,0.646608,baseball_field,0.702366,25_10000
...,...,...,...,...,...,...
147,150,10000,0.641029,transformer_station,0.635342,150_10000
148,25,10000,0.646608,wastewater_treatment_plant,0.582049,25_10000
149,50,10000,0.639308,wastewater_treatment_plant,0.579714,50_10000
150,100,10000,0.639503,wastewater_treatment_plant,0.604003,100_10000


In [1]:
import pandas as pd
df = pd.read_pickle("..\\results\\bovw\\pattern_net\\results_PatternNet.pkl.gz")
df

,model,output_size,anmrr,agricultural,airplane,baseballdiamond,beach,buildings,chaparral,denseresidential,...,intersection,mediumresidential,mobilehomepark,overpass,parkinglot,river,runway,sparseresidential,storagetanks,tenniscourt
0,resnet18,25,0.399225,0.196519,0.257152,0.518165,0.149937,0.604937,0.062278,0.70443,...,0.467089,0.458924,0.520696,0.479873,0.182278,0.565696,0.311962,0.587468,0.632785,0.645190
1,resnet18,50,0.379355,0.232975,0.260570,0.508038,0.121899,0.540759,0.063608,0.65038,...,0.376203,0.451709,0.477405,0.481646,0.174494,0.518481,0.372405,0.596203,0.532911,0.590443
2,resnet50,25,0.326706,0.144367,0.140190,0.400570,0.026835,0.570316,0.180380,0.63057,...,0.441519,0.517785,0.410949,0.438165,0.124937,0.315443,0.239620,0.476203,0.491899,0.524241
3,resnet50,50,0.303888,0.127152,0.098354,0.310000,0.013987,0.467975,0.135316,0.61443,...,0.442025,0.432215,0.279810,0.359810,0.083354,0.433924,0.353987,0.451835,0.450633,0.566519


In [3]:
import plotly.express as px

results_long_form = df.melt(id_vars=['clusters', 'samples', 'anmrr'], var_name='class', value_name='anmrr_per_class')
results_long_form['experiment_name'] = results_long_form.apply(lambda row: str(row['clusters']) + "_" + str(row['samples']), axis=1)
fig = px.bar(results_long_form, x='class', y='anmrr_per_class', color='experiment_name', barmode='group')
fig.show(renderer='browser')

In [ ]:
df = pd.read_pickle("..\\results\\triplet\\uc_merced\\results_uc_merced.pkl.gz")

In [ ]:
import plotly.express as px

results_long_form = df.melt(id_vars=['model', 'output_size', 'anmrr'], var_name='class', value_name='anmrr_per_class')
results_long_form['experiment_name'] = results_long_form.apply(lambda row: str(row['model']) + "_" + str(row['output_size']), axis=1)
fig = px.bar(results_long_form, x='class', y='anmrr_per_class', color='experiment_name', barmode='group')
fig.show(renderer='browser')